# Classification : survivants du Titanic

In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV 

In [3]:
titanic = sns.load_dataset('titanic')
titanic = titanic[['survived','pclass','sex','age']]
titanic['sex'].replace(['male','female'], [0,1], inplace = True)
titanic = titanic.dropna()

In [4]:
model = KNeighborsClassifier()
y = titanic['survived']
X = titanic.drop('survived', axis=1)
model.fit(X,y)
model.score(X,y) # 83% du temps notre modèle arrive à prédire si un passager a survécu ou non

0.8417366946778712

In [5]:
def survie(model, pclass=2, sex =0, age=21 ):
    X = np.array([pclass,sex,age]).reshape(1,3)
    print(model.predict(X))
    print(model.predict_proba(X))
    return None 

survie(model) # je n'aurais pas survécu ...

[0]
[[0.8 0.2]]


/Users/augustincablant/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/Users/augustincablant/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(


## On va maintenant chercher à trouver le meilleur nombre de neighbors

### Méthode naïve

In [18]:
# Méthode naïve : 

best_prediction = 0
indice = 0

for i in range(1,20):
    model = KNeighborsClassifier(n_neighbors = i)
    y = titanic['survived']
    X = titanic.drop('survived', axis=1)
    model.fit(X,y)
    if best_prediction < model.score(X,y):
        best_prediction = model.score(X,y)
        indice = i
    else:
        pass

print("La meilleure prédiction se fait pour", indice, "voisins, et vaut : ", best_prediction)
    

La meilleure prédiction se fait pour 3 voisins, et vaut :  0.8725490196078431


### Méthode efficace

1) Créer un Train set et un Test set, entraîner puis évaluer 
2) Avec GridSearch, trouver les meilleurs hyperparamètres n_neighbors, metric

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)
model = KNeighborsClassifier()
model.fit(X_train,y_train)
model.score(X_test,y_test)

# résultat qui n'est pas ouf 

0.8601398601398601

In [23]:
model = KNeighborsClassifier()
param_grid = {'n_neighbors':np.arange(1,50),
              'metric':['euclidean','manhattan']}
grid = GridSearchCV(KNeighborsClassifier(),param_grid,cv=5)
grid.fit(X_train,y_train)
model = grid.best_estimator_
model.score(X_test,y_test)
print(grid.best_score_, grid.best_params_)

0.7565369946605645 {'metric': 'manhattan', 'n_neighbors': 13}
